In [ ]:
import pandas as pd
import pprint
from messari.messari import Messari
from tabulate import tabulate
from dotenv import load_dotenv
import os

In [ ]:
dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)
messari_api = os.getenv("MESSARI_API")

In [ ]:
messari = Messari(messari_api)
assetsData = messari.get_all_assets(limit=500)["data"]

In [ ]:
columns = list(pd.DataFrame(assetsData).columns)
metrics = list(assetsData[0]["metrics"].keys())
profile = list(assetsData[0]["profile"].keys())

metricsKeys = {}
profileKeys = {}

for i in range(len(metrics)):
    try:
        metricsKeys[metrics[i]] = list(assetsData[0]["metrics"][metrics[i]].keys())
    except:
        pass

for i in range(len(profile)):
    try:
        profileKeys[profile[i]] = list(assetsData[0]["profile"][profile[i]].keys())
    except:
        pass



print(columns)
print("\n")
print(metricsKeys.keys())
print("\n")
print(profileKeys.keys())
#print("\n")
#print(symbols)

In [ ]:
masterlist = []

useMetricKeys = ["market_data", "marketcap", "supply", "token_sale_stats", "developer_activity", "roi_data", "roi_by_year", "risk_metrics", "supply_activity", "supply_distribution"]
useProfKeys = ['economics', 'governance']
# for each asset
for asset in range(len(assetsData)):
    assetdict = {}
    assetdict["name"] =  assetsData[asset]["name"]
    assetdict["symbol"] = assetsData[asset]["symbol"]

    # for each metric in the asset
    for metKey in metricsKeys:
        # if metric is in the list of metrics to use
        if metKey in useMetricKeys:
            # for each column within the metric key
            for k in metricsKeys[metKey]:
                # if column itself has a list of values 
                if type(assetsData[asset]["metrics"][metKey][k]) == dict:
                    # separate the list of values into separate columns
                    for dk in assetsData[asset]["metrics"][metKey][k]:
                        assetdict[k+"__"+dk] = assetsData[asset]["metrics"][metKey][k][dk]
                # otherwise store as column value
                else:
                    assetdict[k] = assetsData[asset]["metrics"][metKey][k]
        else:
            pass

    # for each profile in the asset
    for profKey in profileKeys:
        # if profile is in the list of profiles to use
        if profKey in useProfKeys:
            # for each column within the profile key
            for k in profileKeys[profKey]:
                if k in ["grants"]:
                    pass
                else:
                    # if column itself has a list of values 
                    if type(assetsData[asset]["profile"][profKey][k]) == dict:
                        # separate the list of values into separate columns
                        for dk in assetsData[asset]["profile"][profKey][k]:
                            # if profile:economics:block_explorer don't bother
                            if dk in ["block_explorers", "accounts", "multitoken"]:
                                pass
                            else:
                                #if recursive dictionary do it again
                                if type(assetsData[asset]["profile"][profKey][k][dk]) == dict:
                                    for ddk in assetsData[asset]["profile"][profKey][k][dk]:
                                        assetdict[dk+"__"+ddk] = assetsData[asset]["profile"][profKey][k][dk][ddk]
                                elif type(assetsData[asset]["profile"][profKey][k][dk]) == list:
                                    if len(assetsData[asset]["profile"][profKey][k][dk]) == 0:
                                        assetdict[k+"__"+dk] = None
                                    else:
                                        print(k+"__"+dk, assetsData[asset]["profile"][profKey][k][dk])
                                else:
                                    assetdict[k+"__"+dk] = assetsData[asset]["profile"][profKey][k][dk]

                    # otherwise store as column value
                    elif type(assetsData[asset]["profile"][profKey][k]) == list:
                        if len(assetsData[asset]["profile"][profKey][k]) == 0:
                            assetdict[k] = None
                        else:
                            pass
                    else:
                        assetdict[k] = assetsData[asset]["profile"][profKey][k]
        else:
            pass
    
    # append in master list
    masterlist.append(assetdict)

# store as dataframe
assetdf = pd.DataFrame(masterlist)
assetdf

In [ ]:
# write to csv as well as plaintext
assetdf.to_csv("assets.csv")
with open('table.txt', 'w') as f:
    f.write(tabulate(assetdf, headers='keys', tablefmt='grid'))